In [ ]:
DATA_PATH = "data_with_sentiment_new.xlsx"
OUT_XLSX = "balanced_data_80_balanced.xlsx" 
MODEL_ID = "cointegrated/rut5-base-paraphraser"

TEMPERATURE = 0.9
J_LOWER, J_UPPER = 0.15, 0.85
SLEEP_BETWEEN_GEN = 0.1

from typing import Optional
import re, time, random, pathlib, warnings
import pandas as pd
from tqdm.auto import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

warnings.filterwarnings("ignore", category=FutureWarning)

if not pathlib.Path(DATA_PATH).exists():
    raise FileNotFoundError(f"{DATA_PATH} not found")

df = pd.read_excel(DATA_PATH)[["Review", "Sentiment"]].dropna()

num2str = {1: "positive", -1: "negative", 0: "neutral"}
if df["Sentiment"].dtype != object:
    df["Sentiment"] = df["Sentiment"].map(num2str)

print("Распределение классов до балансировки:\n",
      df["Sentiment"].value_counts(), "\n")

print("Загружаем RuT5-paraphraser …")
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID, torch_dtype=dtype, device_map="auto"
)
paraphraser = pipeline(
    "text2text-generation",
    model=model, tokenizer=tok,
    max_length=128, num_beams=4, repetition_penalty=1.15,
)

def paraphrase(txt: str, T: float = TEMPERATURE) -> Optional[str]:
    try:
        out = paraphraser(
            f"paraphrase: {txt}",
            do_sample=True, top_k=50, top_p=0.95,
            temperature=T, num_return_sequences=1
        )[0]["generated_text"]
        return out.strip(' \n"«»“”')
    except Exception as e:
        print("Paraphrase error:", e)
        return None

def clean(t: str) -> str:
    return re.sub(r"\s+", " ", re.sub(r"<[^>]+>", " ", t)).strip()

def jaccard(a: str, b: str) -> float:
    sa, sb = set(a.lower().split()), set(b.lower().split())
    return 1.0 if not (sa or sb) else len(sa & sb) / len(sa | sb)

cnt = df["Sentiment"].value_counts()
MAX_CLASS = cnt.max()
MINORITY = [cls for cls, n in cnt.items() if n < MAX_CLASS]

synth_rows = []
for label in MINORITY:
    need = MAX_CLASS - cnt[label]
    pool = df[df["Sentiment"] == label]["Review"].tolist()
    print(f"Генерируем для «{label}»: нужно {need}")
    bar = tqdm(total=need, leave=False)
    while need > 0:
        src = clean(random.choice(pool))
        new = paraphrase(src)
        if not new:
            continue
        if not (J_LOWER < jaccard(src, clean(new)) < J_UPPER):
            continue
        if any(jaccard(clean(new), clean(r["Review"])) > J_UPPER
               for r in synth_rows):
            continue
        synth_rows.append({"Review": new, "Sentiment": label})
        need -= 1
        bar.update(1)
        time.sleep(SLEEP_BETWEEN_GEN)
    bar.close()

df_bal = pd.concat([df, pd.DataFrame(synth_rows)], ignore_index=True)
df_bal = df_bal.sample(frac=1, random_state=42).reset_index(drop=True)

df_bal.to_excel(OUT_XLSX, index=False)

print("\nБаланс после генерации:")
print(df_bal["Sentiment"].value_counts())
print(f"Файл сохранён: {OUT_XLSX}")

Распределение классов до балансировки:
 Sentiment
positive    1669
negative     995
neutral      499
Name: count, dtype: int64 

Загружаем RuT5-paraphraser …


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

  0%|          | 0/674 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Генерируем для «neutral»: нужно 1170


  0%|          | 0/1170 [00:00<?, ?it/s]


Баланс после генерации:
Sentiment
neutral     1669
negative    1669
positive    1669
Name: count, dtype: int64
Файл сохранён: balanced_data_80_balanced.xlsx


In [2]:
from google.colab import files
files.download('balanced_data_80_balanced.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>